### MRMR-More

More experiments with the MRMR codebase.

In [1]:
import sys
sys.path.append("../..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import pathlib
import yaml
import tqdm
from wbf_simulate import run_nrobot1day


In [2]:
# Run an experiment with a different budget
experiment_path = pathlib.Path(Config()["experiment_external"])
experiment_path.mkdir(exist_ok=True)
experiment_data = pathlib.Path(Config()["data_external"])
experiment_data.mkdir(exist_ok=True)

Config().set_experiment_path(experiment_path)
Config().set_experiment_data(experiment_data)
# FIXME this is dangerous, because if it has the original paths, it overwrites the experiments, this needs to be fixed
Config().copy_experiment("mrmr")
Config().copy_experiment("environment")
Config().copy_experiment("policy")
Config().copy_experiment("estimator")
Config().copy_experiment("score")



***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\WaterBerryFarms\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\WBF-config-LotziYoga.yaml
***ExpRun**: Experiment config path changed to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external
***ExpRun**: Experiment data path changed to c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external
***ExpRun**: Experiment mrmr copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\mrmr
***ExpRun**: Experiment environment copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\environment
***ExpRun**: Experiment policy copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\policy
***ExpRun**: Experiment estimator copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\estimator
***ExpRun**: Experiment score copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\s

In [3]:
def generate_rw(experiment="mrmr", n_robots = 1, budget = 1000, environment = "mrmr-custom-notclustered"):
    """Generate experiment with n robots following random waypoint, and save it to the experiment path"""
    runname = f"gen-rw-n_{n_robots}-b_{budget}-e_{environment}"
    val = {}    
    val["name"] = "n random waypoint"

    val["exp_environment"] = "environment"
    val["run_environment"] = environment

    val["exp_estimator"] = "estimator"
    val["run_estimator"] = "adaptive-disk"

    val["exp_score"] = "score"
    val["run_score"] = "weighted_asymmetric"

    val["timesteps-per-day"] = budget
    val["time-start-environment"] = 19

    robots = []
    val["robots"] = robots

    for robot in range(n_robots):
        v = {}
        robots.append(v)
        v["name"] = f"robot-{robot}"
        v["exp-policy"] = "policy"
        v["run-policy"] = "random-waypoint"
        vextra = {}
        v["exp-policy-extra-parameters"] = vextra
        vextra["seed"] = robot
        vextra["policy-name"] = f"rwp-{robot}"
        vextra["budget"] = budget
    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), experiment, runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    return runname

In [4]:
def generate_lawnmowers(experiment = "mrmr", n_robots = 1, budget = 1000, environment = "mrmr-custom-notclustered"):
    """Generate experiment with n robots following lawnmower, with the area being divided into n areas"""
    runname = f"gen-lawn-n_{n_robots}-b_{budget}-e_{environment}"
    val = {}    
    val["name"] = "n random waypoint"

    val["exp_environment"] = "environment"
    val["run_environment"] = environment

    val["exp_estimator"] = "estimator"
    val["run_estimator"] = "adaptive-disk"

    val["exp_score"] = "score"
    val["run_score"] = "weighted_asymmetric"

    val["timesteps-per-day"] = budget
    val["time-start-environment"] = 19

    robots = []
    val["robots"] = robots

    for robot in range(n_robots):
        v = {}
        robots.append(v)
        v["name"] = f"robot-{robot}"
        v["exp-policy"] = "policy"
        v["run-policy"] = "fixed-budget-lawnmower"
        vextra = {}
        v["exp-policy-extra-parameters"] = vextra
        area = [0,0,0,0]
        border = 5
        width = (100 - 2 * border) / n_robots
        area[0] = int(border)
        area[1] = int(border + robot * width)
        area[2] = 100 - border
        area[3] = int(border + (robot+1) * width)
        vextra["area"] = str(area)
        vextra["policy-name"] = f"lawn-{robot}"
        vextra["budget"] = budget

    # save the generated path 
    path = pathlib.Path(Config().get_experiment_path(), experiment, runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    return runname

In [5]:
def generate_mrmr(experiment="mrmr", n_robots = 1, budget = 1000, environment = "mrmr-custom-notclustered"):
    """Generate experiment with n robots following lawnmower, with the area being divided into n areas"""
    runname = f"gen-mrmr_{n_robots}-b_{budget}-e_{environment}"
    val = {}    
    val["name"] = "n random waypoint"

    val["exp_environment"] = "environment"
    val["run_environment"] = environment

    val["exp_estimator"] = "estimator"
    val["run_estimator"] = "adaptive-disk"

    val["exp_score"] = "score"
    val["run_score"] = "weighted_asymmetric"

    val["timesteps-per-day"] = budget
    val["time-start-environment"] = 19

    robots = []
    val["robots"] = robots

    # generate the first robot, the pioneer
    v = {}
    robots.append(v)
    v["name"] = f"pio"
    v["exp-policy"] = "policy"
    v["run-policy"] = "generic"    
    vextra = {}
    v["exp-policy-extra-parameters"] = vextra
    vextra["policy-name"] = f"mrmr-pioneer"
    vextra["policy-code"] = "MRMR_Pioneer"
    vextra["seed"] = 1
    vextra["budget"] = budget

    # generate the other robots, the contractors
    for robot in range(n_robots-1):
        v = {}
        robots.append(v)
        v["name"] = f"con-{robot}"
        v["exp-policy"] = "policy"
        v["run-policy"] = "generic"
        vextra = {}
        v["exp-policy-extra-parameters"] = vextra
        vextra["seed"] = robot + 1
        vextra["policy-name"] = f"mrmr-contractor-{robot}"
        vextra["policy-code"] = "MRMR_Contractor"
        vextra["budget"] = budget

    # save the generated path 
    path = pathlib.Path(Config().get_experiment_path(), experiment, runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    return runname

### Generate a range of experiments

In [6]:
# run = generate_rw(n_robots=3, budget=1000)
# run = generate_lawnmowers(n_robots=3, budget=1000)
runs = []
# the runs of the initial submission
runs_1000_clustered = []
runs_1000_notclustered = []
# FIXME: generate some other lists as well
experiment = "mrmr"

for n_robots in range(1, 5):
    #for budget in [500, 1000, 1500, 2000]:
    for budget in [500, 1000, 1500]:
        for environment in ["mrmr-custom-notclustered", "mrmr-custom-clustered"]:
            runs2 = []
            run = generate_lawnmowers(experiment, n_robots=n_robots, budget=budget, environment = environment)
            runs2.append(run)
            # 
            run = generate_lawnmowers(experiment, n_robots=n_robots, budget=budget, environment = environment)
            runs2.append(run)
            #
            run = generate_mrmr(experiment, n_robots=n_robots, budget=budget, environment = environment)
            runs2.append(run)
            # 
            runs.extend(runs2)
            if budget == 1000 and environment == "mrmr-custom-notclustered":
                runs_1000_notclustered.extend(runs2)
            if budget == 1000 and environment == "mrmr-custom-clustered":
                runs_1000_clustered.extend(runs2)                

# Run the experiments

In [7]:
for run in tqdm.tqdm(runs):
    exp = Config().get_experiment("mrmr", run)
    run_nrobot1day(exp)

  0%|          | 0/72 [00:00<?, ?it/s]

***ExpRun**: Configuration for exp/run: mrmr/gen-lawn-n_1-b_500-e_mrmr-custom-notclustered successfully loaded
Results file already exists:
c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\mrmr\gen-lawn-n_1-b_500-e_mrmr-custom-notclustered\results.pickle
Delete this file if re-running is desired.
***ExpRun**: Configuration for exp/run: mrmr/gen-lawn-n_1-b_500-e_mrmr-custom-notclustered successfully loaded
Results file already exists:
c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\mrmr\gen-lawn-n_1-b_500-e_mrmr-custom-notclustered\results.pickle
Delete this file if re-running is desired.
***ExpRun**: Configuration for exp/run: mrmr/gen-mrmr_1-b_500-e_mrmr-custom-notclustered successfully loaded
Results file already exists:
c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\mrmr\gen-mrmr_1-b_500-e_mrmr-custom-notclustered\results.pickle
Delete this file if re-running is desired.
***ExpRun**: Configuration for exp/run: mrmr/gen-lawn-n_1-b_500-e_mrmr-custom

 26%|██▋       | 19/72 [00:00<00:00, 188.43it/s]

Results file already exists:
c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\mrmr\gen-lawn-n_2-b_500-e_mrmr-custom-notclustered\results.pickle
Delete this file if re-running is desired.
***ExpRun**: Configuration for exp/run: mrmr/gen-lawn-n_2-b_500-e_mrmr-custom-notclustered successfully loaded
Results file already exists:
c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\mrmr\gen-lawn-n_2-b_500-e_mrmr-custom-notclustered\results.pickle
Delete this file if re-running is desired.
***ExpRun**: Configuration for exp/run: mrmr/gen-mrmr_2-b_500-e_mrmr-custom-notclustered successfully loaded
Results file already exists:
c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\mrmr\gen-mrmr_2-b_500-e_mrmr-custom-notclustered\results.pickle
Delete this file if re-running is desired.
***ExpRun**: Configuration for exp/run: mrmr/gen-lawn-n_2-b_500-e_mrmr-custom-clustered successfully loaded
Results file already exists:
c:\Users\lboloni\Documents\Code\_TempData\WBF-data-

 53%|█████▎    | 38/72 [00:00<00:00, 189.29it/s]

...

INFO:root:create_type_map shape=(10201,)
INFO:root:Infection matrix: [0.         0.25       0.4375     0.578125   0.68359375 0.76269531
 0.82202148 0.86651611 0.89988708 0.92491531 0.94368649 0.95776486
 0.96832365 0.97624274 0.98218205 0.98663654 0.9899774  0.99248305
 0.99436229 0.99577172 0.99682879 0.99762159 0.99821619 0.99866214
 0.99899661]
INFO:root:Infection matrix: [0.         0.15       0.2775     0.385875   0.47799375 0.55629469
 0.62285048 0.67942291 0.72750947 0.76838305 0.8031256  0.83265676
 0.85775824 0.87909451 0.89723033 0.91264578 0.92574891 0.93688658
 0.94635359 0.95440055 0.96124047 0.9670544  0.97199624 0.9761968
 0.97976728]
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 1
INFO:root:PrecalculatedEnvironment: file c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\environment\mrmr-custom-clustered\precalc_tylcv\env_value_00001.gz already 

budget
policy-code
policy-name
seed
[{'exp-policy': Experiment:
    budget: 1500
    data_dir: c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\policy\generic
    exp_run_sys_indep_file: c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\policy\generic.yaml
    experiment_name: policy
    policy-code: MRMR_Pioneer
    policy-name: mrmr-pioneer
    run_name: generic
    seed: 1
    subrun_name: null
,
  'name': 'pio'},
 {'exp-policy': Experiment:
    budget: 1500
    data_dir: c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\policy\generic
    exp_run_sys_indep_file: c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\policy\generic.yaml
    experiment_name: policy
    policy-code: MRMR_Contractor
    policy-name: mrmr-contractor-0
    run_name: generic
    seed: 1
    subrun_name: null
,
  'name': 'con-0'},
 {'exp-policy': Experiment:
    budget: 1500
    data_dir: c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\policy\generic
    exp_ru

INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 20
INFO:root:PrecalculatedEnvironment: file c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\environment\mrmr-custom-clustered\precalc_tylcv\env_value_00020.gz already exists, skipping!
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 20
INFO:root:PrecalculatedEnvironment: file c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\environment\mrmr-custom-clustered\precalc_ccr\env_value_00020.gz already exists, skipping!
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 20
INFO:root:PrecalculatedEnvironment: file c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\environment\mrmr-custom-clustered\precalc_soil\env_value_00020.gz already exists, skipping!


Part three first step 0, 0, 0.0
Part three last step {'t': 1499.0, 'x': 19.0, 'y': 51.0, 'ep': None}
Part three first step 0, 0, 0.0
Part three last step {'t': 1499.0, 'x': 54.92941911250381, 'y': 17.187827523724167, 'ep': None}
Part three first step 0, 0, 0.0
Part three last step {'t': 1499.0, 'x': 31.435745612917945, 'y': 56.814743983477, 'ep': None}
Part two first step 39.99308755658387, 41.647329899741834, 211.0
Part two last step {'t': 279.0, 'x': 30.0, 'y': 36.0, 'ep': ExplorationPackage x=[30,41] y=[38, 44] step=2}
Part three first step 30.0, 36.0, 280.0
Part three last step {'t': 1499.0, 'x': 61.768539925326195, 'y': 23.31585551407011, 'ep': None}
Self plan first {'t': 211.0, 'x': 39.99308755658387, 'y': 41.647329899741834, 'ep': None}, 
 robot location 39.99308755658387,41.647329899741834 
 last observation {'x': 39, 'y': 41, 'time': 210, 'TYLCV': {'x': 39, 'y': 41, 'time': 210, 'value': np.float64(1.0)}, 'CCR': {'x': 39, 'y': 41, 'time': 210, 'value': np.float64(0.0)}, 'Soil'

100%|██████████| 72/72 [07:30<00:00,  6.26s/it]
